# 03 - Simulação de Interações (BKT)

Este notebook implementa a **Etapa 3** do pipeline SINKT: geração de interações simuladas usando apenas o modelo BKT.

## Objetivo
Gerar sequências de interações (acertos/erros) para cada estudante baseado nos parâmetros cognitivos e BKT, sem geração de respostas textuais.

## Saída
- `data/output/notebooks/simulacao_interacoes/interactions_bkt.json`: Arquivo JSON contendo todas as interações simuladas (sem respostas LLM)

## Importação de Bibliotecas

In [23]:
import json
import os
from datetime import datetime, timedelta
from typing import Dict, List, Any, Tuple
import numpy as np
import pandas as pd
from collections import defaultdict, Counter
import random
import time

print("✅ Bibliotecas importadas com sucesso")

✅ Bibliotecas importadas com sucesso


## Carregamento de Dados

In [24]:
with open('data/output/notebooks/geracao_perfis/profiles.json', 'r', encoding='utf-8') as f:
    profiles_data = json.load(f)
profiles = profiles_data['profiles']

with open('data/output/notebooks/geracao_estudantes/students.json', 'r', encoding='utf-8') as f:
    students_data = json.load(f)
students = students_data['students']

with open('data/json/questions_graph.json', 'r', encoding='utf-8') as f:
    questions_data = json.load(f)
questions = questions_data.get('questions', [])

with open('data/json/concepts_graph.json', 'r', encoding='utf-8') as f:
    concepts_data = json.load(f)
concepts = concepts_data.get('concepts', [])

print(f"✅ Dados carregados:")
print(f"  - Perfis: {len(profiles)}")
print(f"  - Estudantes: {len(students)}")
print(f"  - Questões: {len(questions)}")
print(f"  - Conceitos: {len(concepts)}")

✅ Dados carregados:
  - Perfis: 6
  - Estudantes: 100
  - Questões: 680
  - Conceitos: 251


## Configuração de Parâmetros

In [25]:
MIN_INTERACTIONS_PER_STUDENT = 30
MAX_INTERACTIONS_PER_STUDENT = 60
SEED = 42

CHECKPOINT_INTERVAL = 10
CHECKPOINT_FILE = 'data/output/notebooks/simulacao_interacoes/interactions_bkt_checkpoint.json'

ERROR_TYPES = [
    'misconception',
    'careless',
    'slip',
    'incomplete',
    'misunderstanding'
]

np.random.seed(SEED)
random.seed(SEED)

print(f"✅ Configuracoes:")
print(f"  - Interacoes por estudante: {MIN_INTERACTIONS_PER_STUDENT}-{MAX_INTERACTIONS_PER_STUDENT}")
print(f"  - Checkpoint a cada: {CHECKPOINT_INTERVAL} estudantes")
print(f"  - Seed: {SEED}")

✅ Configuracoes:
  - Interacoes por estudante: 30-60
  - Checkpoint a cada: 10 estudantes
  - Seed: 42


## Funcoes BKT

- Usa `current_mastery` na probabilidade (nao `mastery_init_level`)
- Bayesiano padrao seguido de transicao de aprendizagem
- Fatores cognitivos afetam `slip_eff` e `guess_eff`
- Ruido de consistencia aplicado na probabilidade final
- Decay temporal so com gap > 6h, coeficiente reduzido

In [26]:
# Constantes para ajuste fino dos pesos cognitivos
DECAY_THRESHOLD_SECONDS = 6 * 3600  # 6 horas em segundos
DECAY_COEFFICIENT = 0.02  # Reduzido de 0.1

def calculate_effective_params(student_params: Dict, question_difficulty: float) -> Tuple[float, float, float]:
    """
    Calcula slip, guess e learn_rate efetivos baseado nos fatores cognitivos.
    
    Correção C: Fatores cognitivos afetam slip/guess, não mastery diretamente.
    """
    # Parâmetros base do estudante
    slip_base = student_params.get('slip', 0.1)
    guess_base = student_params.get('guess', 0.15)
    learn_rate_base = student_params.get('learn_rate', 0.05)
    
    # Fatores cognitivos
    logic_skill = student_params.get('logic_skill', 0.5)
    reading_skill = student_params.get('reading_skill', 0.5)
    tech_familiarity = student_params.get('tech_familiarity', 0.5)
    learning_consistency = student_params.get('learning_consistency', 0.5)
    
    # guess_eff: tech e logic aumentam chance de "se virar" mesmo sem dominar
    # Dificuldade alta reduz o guess efetivo
    guess_eff = guess_base * (0.85 + 0.20 * tech_familiarity + 0.15 * logic_skill - 0.20 * question_difficulty)
    guess_eff = np.clip(guess_eff, 0.0, 0.35)
    
    # slip_eff: baixa consistencia e reading aumentam erro tipo "descuido/misunderstanding"
    # Dificuldade alta aumenta slip
    slip_eff = slip_base * (0.90 + 0.40 * (1 - learning_consistency) + 0.20 * question_difficulty + 0.20 * (1 - reading_skill))
    slip_eff = np.clip(slip_eff, 0.0, 0.45)
    
    # learn_eff: tech ajuda a converter tentativa em aprendizado
    learn_eff = learn_rate_base * (0.85 + 0.15 * tech_familiarity)
    learn_eff = np.clip(learn_eff, 0.0, 0.35)
    
    return slip_eff, guess_eff, learn_eff

def calculate_response_probability(current_mastery: float, student_params: Dict, 
                                   question_difficulty: float) -> Tuple[float, float, float]:
    """
    Calcula probabilidade de resposta correta usando formula BKT classica.
    
    Formula BKT: P(correct) = P(L)*(1-slip) + (1-P(L))*guess
    
    Usa current_mastery (nao mastery_init_level)
    Fatores cognitivos afetam slip/guess efetivos
    Ruido de consistencia aplicado na prob final, nao no mastery
    """
    # Calcula parametros efetivos baseado em fatores cognitivos
    slip_eff, guess_eff, learn_eff = calculate_effective_params(student_params, question_difficulty)
    
    # Ajuste leve do mastery por dificuldade (pequeno efeito)
    k_d = 0.15  # Coeficiente de dificuldade
    m_adj = np.clip(current_mastery - k_d * question_difficulty, 0.0, 1.0)
    
    # Formula BKT classica: P(correct) = P(L)*(1-slip) + (1-P(L))*guess
    prob = m_adj * (1 - slip_eff) + (1 - m_adj) * guess_eff
    
    # Ruido de consistencia na probabilidade final (nao no mastery)
    learning_consistency = student_params.get('learning_consistency', 0.5)
    if learning_consistency < 0.7:
        noise_sigma = (0.7 - learning_consistency) * 0.1  # Ruido proporcional a inconsistencia
        noise = np.random.normal(0, noise_sigma)
        prob += noise
    
    prob = np.clip(prob, 0.0, 1.0)
    
    return prob, slip_eff, guess_eff

def update_mastery_bkt(current_mastery: float, is_correct: bool, 
                       slip: float, guess: float, learn_rate: float,
                       memory_capacity: float = 0.5, time_gap: float = 0) -> float:
    """
    Atualiza o dominio do estudante usando update BKT padrao (Bayes + learn).
    
    Update Bayesiano correto seguido de transicao de aprendizagem.
    Decay temporal so com gap > 6h, coeficiente reduzido.
    
    Referencia: Corbett & Anderson (1995) - Knowledge Tracing
    """
    P_L = current_mastery
    
    # Update Bayesiano dado observacao (acerto/erro)
    if is_correct:
        # P(L|correct) = P(L)*(1-slip) / [P(L)*(1-slip) + (1-P(L))*guess]
        numerator = P_L * (1 - slip)
        denominator = P_L * (1 - slip) + (1 - P_L) * guess
        if denominator > 0:
            P_L_given_obs = numerator / denominator
        else:
            P_L_given_obs = P_L
    else:
        # P(L|wrong) = P(L)*slip / [P(L)*slip + (1-P(L))*(1-guess)]
        numerator = P_L * slip
        denominator = P_L * slip + (1 - P_L) * (1 - guess)
        if denominator > 0:
            P_L_given_obs = numerator / denominator
        else:
            P_L_given_obs = P_L
    
    # Transicao de aprendizagem: P(L_next) = P(L|obs) + (1 - P(L|obs)) * learn_rate
    P_L_next = P_L_given_obs + (1 - P_L_given_obs) * learn_rate
    
    # Correcao E: Decay temporal so se gap > 6h (DECAY_THRESHOLD_SECONDS)
    if time_gap > DECAY_THRESHOLD_SECONDS:
        # Decay proporcional a (1 - memory_capacity) e ao tempo
        time_factor = min((time_gap - DECAY_THRESHOLD_SECONDS) / 86400, 1.0)  # Normalizado por 24h
        decay_factor = 1 - (1 - memory_capacity) * DECAY_COEFFICIENT * time_factor
        decay_factor = max(0.5, decay_factor)  # Nunca decai mais que 50%
        P_L_next *= decay_factor
    
    return np.clip(P_L_next, 0.0, 1.0)

In [27]:
interactions = []
start_time = time.time()

for idx, student in enumerate(students, 1):
    student_id = student['id']
    num_interactions = np.random.randint(MIN_INTERACTIONS_PER_STUDENT, MAX_INTERACTIONS_PER_STUDENT + 1)
    
    # Inicializa com mastery_init_level, mas usa current_mastery na probabilidade
    current_mastery = student.get('mastery_init_level', 0.5)
    last_interaction_time = datetime.now() - timedelta(days=30)
    
    # Calcula learn_rate efetivo uma vez por estudante (baseado em tech_familiarity)
    _, _, learn_eff = calculate_effective_params(student, 0.5)
    
    for i in range(num_interactions):
        question = random.choice(questions)
        question_id = question['id']
        question_difficulty = question.get('difficulty', 0.5)
        
        time_gap = (datetime.now() - last_interaction_time).total_seconds()
        
        # Passa current_mastery (nao mastery_init_level)
        prob_correct, slip_eff, guess_eff = calculate_response_probability(
            current_mastery, student, question_difficulty
        )
        # Converte np.bool_ para bool Python nativo (JSON serializable)
        is_correct = bool(np.random.random() < prob_correct)
        
        error_type = None
        if not is_correct:
            error_type = random.choice(ERROR_TYPES)
        
        interaction = {
            'student_id': student_id,
            'question_id': question_id,
            'is_correct': is_correct,
            'mastery_before': round(float(current_mastery), 4),
            'probability': round(float(prob_correct), 4),
            'slip_eff': round(float(slip_eff), 4),
            'guess_eff': round(float(guess_eff), 4),
            'error_type': error_type,
            'timestamp': (datetime.now() - timedelta(days=30-i)).isoformat()
        }
        
        # Update BKT padrao (Bayes + learn)
        current_mastery = update_mastery_bkt(
            current_mastery,
            is_correct,
            slip_eff,
            guess_eff,
            learn_eff,
            student.get('memory_capacity', 0.5),
            time_gap
        )
        
        interaction['mastery_after'] = round(float(current_mastery), 4)
        interactions.append(interaction)
        
        last_interaction_time = datetime.now() - timedelta(days=30-i)
    
    if idx % CHECKPOINT_INTERVAL == 0:
        checkpoint_data = {
            'processed_students': idx,
            'total_interactions': len(interactions),
            'interactions': interactions
        }
        os.makedirs(os.path.dirname(CHECKPOINT_FILE), exist_ok=True)
        with open(CHECKPOINT_FILE, 'w', encoding='utf-8') as f:
            json.dump(checkpoint_data, f, indent=2, ensure_ascii=False)
        print(f"Checkpoint: {idx}/{len(students)} estudantes processados ({len(interactions)} interacoes)")

elapsed_time = time.time() - start_time
print(f"\n✅ Geracao de interacoes concluida!")
print(f"  - Tempo total: {elapsed_time:.2f}s")
print(f"  - Total de interacoes: {len(interactions)}")
print(f"  - Media por estudante: {len(interactions)/len(students):.1f}")

Checkpoint: 10/100 estudantes processados (423 interacoes)
Checkpoint: 20/100 estudantes processados (919 interacoes)
Checkpoint: 30/100 estudantes processados (1345 interacoes)
Checkpoint: 40/100 estudantes processados (1777 interacoes)
Checkpoint: 50/100 estudantes processados (2257 interacoes)


Checkpoint: 60/100 estudantes processados (2738 interacoes)
Checkpoint: 70/100 estudantes processados (3149 interacoes)
Checkpoint: 80/100 estudantes processados (3579 interacoes)
Checkpoint: 90/100 estudantes processados (4059 interacoes)
Checkpoint: 100/100 estudantes processados (4499 interacoes)

✅ Geracao de interacoes concluida!
  - Tempo total: 0.30s
  - Total de interacoes: 4499
  - Media por estudante: 45.0


## Salvamento das Interacoes

In [28]:
output_data = {
    "metadata": {
        "description": "Interacoes simuladas com BKT corrigido (Corbett & Anderson, 1995)",
        "version": "4.0.0",
        "created_at": datetime.now().isoformat(),
        "total_interactions": len(interactions),
        "total_students": len(students),
        "avg_interactions_per_student": len(interactions) / len(students),
        "seed": SEED,
        "bkt_corrections": {
            "A": "Usa current_mastery na probabilidade (nao mastery_init)",
            "B": "Update Bayesiano padrao + transicao de aprendizagem",
            "C": "Fatores cognitivos afetam slip/guess efetivos",
            "D": "Ruido de consistencia na prob final (nao no mastery)",
            "E": "Decay temporal so com gap > 6h, coeficiente reduzido"
        }
    },
    "interactions": interactions
}

output_file = 'data/output/notebooks/simulacao_interacoes/interactions_bkt.json'
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

if os.path.exists(CHECKPOINT_FILE):
    os.remove(CHECKPOINT_FILE)
    print("Checkpoint removido (processamento completo)")

print(f"\n✅ Interacoes salvas em: {output_file}")
print(f"Total de interacoes: {len(interactions)}")
print(f"Tamanho do arquivo: {os.path.getsize(output_file) / (1024*1024):.2f} MB")

Checkpoint removido (processamento completo)

✅ Interacoes salvas em: data/output/notebooks/simulacao_interacoes/interactions_bkt.json
Total de interacoes: 4499
Tamanho do arquivo: 1.45 MB


## Análise de Métricas

### Estatísticas Gerais

In [29]:
def analyze_general_statistics(interactions: List[Dict]) -> Dict[str, Any]:
    """Calcula estatísticas gerais das interações."""
    total_interactions = len(interactions)
    correct_interactions = sum(1 for i in interactions if i['is_correct'])
    
    stats = {
        'total_interactions': total_interactions,
        'correct_interactions': correct_interactions,
        'incorrect_interactions': total_interactions - correct_interactions,
        'accuracy': correct_interactions / total_interactions if total_interactions > 0 else 0
    }
    
    masteries = [i['mastery_after'] for i in interactions]
    stats['mastery_statistics'] = {
        'mean': np.mean(masteries),
        'median': np.median(masteries),
        'std': np.std(masteries),
        'min': np.min(masteries),
        'max': np.max(masteries)
    }
    
    return stats

general_stats = analyze_general_statistics(interactions)

print("\n✅ Estatísticas Gerais das Interações:\n")
print(f"  Total de Interações: {general_stats['total_interactions']}")
print(f"  Corretas: {general_stats['correct_interactions']} ({general_stats['accuracy']:.1%})")
print(f"  Incorretas: {general_stats['incorrect_interactions']} ({1-general_stats['accuracy']:.1%})")
print(f"\n  Mastery (Domínio):")
print(f"    Média: {general_stats['mastery_statistics']['mean']:.3f}")
print(f"    Mediana: {general_stats['mastery_statistics']['median']:.3f}")
print(f"    Desvio: {general_stats['mastery_statistics']['std']:.3f}")
print(f"    Range: [{general_stats['mastery_statistics']['min']:.3f}, {general_stats['mastery_statistics']['max']:.3f}]")


✅ Estatísticas Gerais das Interações:

  Total de Interações: 4499
  Corretas: 1860 (41.3%)
  Incorretas: 2639 (58.7%)

  Mastery (Domínio):
    Média: 0.484
    Mediana: 0.250
    Desvio: 0.442
    Range: [0.018, 1.000]


### Distribuição de Erros

In [30]:
def analyze_error_distribution(interactions: List[Dict]) -> Dict[str, Any]:
    """Analisa distribuição de tipos de erro."""
    error_distribution = Counter()
    
    for interaction in interactions:
        if interaction['error_type']:
            error_distribution[interaction['error_type']] += 1
    
    total_errors = sum(error_distribution.values())
    
    error_stats = {}
    for error_type, count in error_distribution.items():
        error_stats[error_type] = {
            'count': count,
            'percentage': (count / total_errors * 100) if total_errors > 0 else 0
        }
    
    return {
        'total_errors': total_errors,
        'error_distribution': error_stats
    }

error_analysis = analyze_error_distribution(interactions)

print("\n✅ Análise de Distribuição de Erros:\n")
print(f"  Total de Erros: {error_analysis['total_errors']}")
print(f"\n  Distribuição por Tipo:")
for error_type, stats in error_analysis['error_distribution'].items():
    print(f"    - {error_type}: {stats['count']} ({stats['percentage']:.1f}%)")


✅ Análise de Distribuição de Erros:

  Total de Erros: 2639

  Distribuição por Tipo:
    - misconception: 524 (19.9%)
    - slip: 547 (20.7%)
    - careless: 515 (19.5%)
    - misunderstanding: 516 (19.6%)
    - incomplete: 537 (20.3%)


### Desempenho por Perfil

In [31]:
def analyze_performance_by_profile(interactions: List[Dict], students: List) -> Dict[str, Any]:
    """Analisa desempenho agrupado por perfil cognitivo."""
    profile_data = defaultdict(lambda: {
        'students': [],
        'accuracies': [],
        'masteries': []
    })
    
    student_interactions = defaultdict(list)
    for interaction in interactions:
        student_interactions[interaction['student_id']].append(interaction)
    
    for student in students:
        student_id = student['id']
        profile_id = student['profile_id']
        student_ints = student_interactions.get(student_id, [])
        
        if not student_ints:
            continue
        
        accuracy = sum(1 for i in student_ints if i['is_correct']) / len(student_ints)
        avg_mastery = np.mean([i['mastery_after'] for i in student_ints])
        
        profile_data[profile_id]['students'].append(student_id)
        profile_data[profile_id]['accuracies'].append(accuracy)
        profile_data[profile_id]['masteries'].append(avg_mastery)
    
    profile_stats = {}
    for profile_id, data in profile_data.items():
        if data['accuracies']:
            profile_stats[profile_id] = {
                'num_students': len(data['students']),
                'accuracy': {
                    'mean': np.mean(data['accuracies']),
                    'std': np.std(data['accuracies'])
                },
                'mastery': {
                    'mean': np.mean(data['masteries']),
                    'std': np.std(data['masteries'])
                }
            }
    
    return profile_stats

profile_performance = analyze_performance_by_profile(interactions, students)

print("\n✅ Desempenho por Perfil Cognitivo:\n")
for profile_id, stats in sorted(profile_performance.items()):
    print(f"  {profile_id}:")
    print(f"    Estudantes: {stats['num_students']}")
    print(f"    Acurácia: {stats['accuracy']['mean']:.1%} (±{stats['accuracy']['std']:.1%})")
    print(f"    Domínio: {stats['mastery']['mean']:.3f} (±{stats['mastery']['std']:.3f})")
    print()


✅ Desempenho por Perfil Cognitivo:

  balanced:
    Estudantes: 30
    Acurácia: 43.0% (±24.6%)
    Domínio: 0.499 (±0.319)

  careful:
    Estudantes: 20
    Acurácia: 42.5% (±35.6%)
    Domínio: 0.457 (±0.408)

  intuitive:
    Estudantes: 10
    Acurácia: 37.7% (±24.6%)
    Domínio: 0.419 (±0.315)

  logical:
    Estudantes: 10
    Acurácia: 30.5% (±26.8%)
    Domínio: 0.401 (±0.336)

  quick_learner:
    Estudantes: 20
    Acurácia: 49.5% (±24.2%)
    Domínio: 0.657 (±0.304)

  struggling:
    Estudantes: 10
    Acurácia: 25.0% (±10.1%)
    Domínio: 0.186 (±0.152)



### Correlação entre Parâmetros e Desempenho

In [32]:
def analyze_parameter_correlations(interactions: List[Dict], students: List) -> Dict[str, Any]:
    """Analisa correlação entre parâmetros dos estudantes e seu desempenho."""
    student_interactions = defaultdict(list)
    for interaction in interactions:
        student_interactions[interaction['student_id']].append(interaction)
    
    data = []
    for student in students:
        student_id = student['id']
        student_ints = student_interactions.get(student_id, [])
        if not student_ints:
            continue
        
        accuracy = sum(1 for i in student_ints if i['is_correct']) / len(student_ints)
        avg_mastery = np.mean([i['mastery_after'] for i in student_ints])
        
        data.append({
            'student_id': student_id,
            'accuracy': accuracy,
            'avg_mastery': avg_mastery,
            'learn_rate': student.get('learn_rate', 0),
            'logic_skill': student.get('logic_skill', 0),
            'reading_skill': student.get('reading_skill', 0),
            'tech_familiarity': student.get('tech_familiarity', 0),
            'memory_capacity': student.get('memory_capacity', 0),
            'learning_consistency': student.get('learning_consistency', 0),
            'mastery_init': student.get('mastery_init_level', 0),
            'slip': student.get('slip', 0),
            'guess': student.get('guess', 0)
        })
    
    df = pd.DataFrame(data)
    
    correlations = {}
    for param in ['learn_rate', 'logic_skill', 'reading_skill', 'tech_familiarity',
                  'memory_capacity', 'learning_consistency', 'mastery_init', 'slip', 'guess']:
        if param in df.columns:
            corr_with_accuracy = df[param].corr(df['accuracy'])
            corr_with_mastery = df[param].corr(df['avg_mastery'])
            correlations[param] = {
                'correlation_with_accuracy': round(corr_with_accuracy, 3),
                'correlation_with_mastery': round(corr_with_mastery, 3)
            }
    
    sorted_by_accuracy = sorted(
        correlations.items(),
        key=lambda x: abs(x[1]['correlation_with_accuracy']),
        reverse=True
    )
    
    return {
        'correlations': dict(sorted_by_accuracy),
        'top_5_factors': [f[0] for f in sorted_by_accuracy[:5]]
    }

correlations = analyze_parameter_correlations(interactions, students)

print("\n✅ Correlação entre Parâmetros e Desempenho:\n")
print("  Parâmetro            | Corr. Acurácia | Corr. Domínio")
print("  " + "-" * 60)
for param, corrs in list(correlations['correlations'].items()):
    acc_corr = corrs['correlation_with_accuracy']
    mas_corr = corrs['correlation_with_mastery']
    print(f"  {param:20s} | {acc_corr:>6.3f}         | {mas_corr:>6.3f}")

print(f"\n  Top 5 Fatores Mais Importantes:")
for i, factor in enumerate(correlations['top_5_factors'], 1):
    print(f"    {i}. {factor}")


✅ Correlação entre Parâmetros e Desempenho:

  Parâmetro            | Corr. Acurácia | Corr. Domínio
  ------------------------------------------------------------
  memory_capacity      |  0.252         |  0.345
  mastery_init         |  0.210         |  0.290
  tech_familiarity     |  0.201         |  0.328
  learn_rate           |  0.199         |  0.322
  reading_skill        |  0.170         |  0.165
  learning_consistency |  0.167         |  0.219
  logic_skill          |  0.086         |  0.192
  slip                 | -0.058         | -0.054
  guess                | -0.046         | -0.127

  Top 5 Fatores Mais Importantes:
    1. memory_capacity
    2. mastery_init
    3. tech_familiarity
    4. learn_rate
    5. reading_skill


## Resumo da Execução

In [33]:
print("\n" + "="*70)
print("✅ SIMULAÇÃO DE INTERAÇÕES BKT CONCLUÍDA COM SUCESSO!")
print("="*70)
print(f"\n  Arquivo gerado: {output_file}")
print(f"\n  Resumo:")
print(f"    - Total de interações: {len(interactions)}")
print(f"    - Estudantes: {len(students)}")
print(f"    - Acurácia: {general_stats['accuracy']:.1%}")
print("="*70)


✅ SIMULAÇÃO DE INTERAÇÕES BKT CONCLUÍDA COM SUCESSO!

  Arquivo gerado: data/output/notebooks/simulacao_interacoes/interactions_bkt.json

  Resumo:
    - Total de interações: 4499
    - Estudantes: 100
    - Acurácia: 41.3%

✅ Próximo passo: Execute o notebook '04_geracao_respostas_llm.ipynb'
